In [ ]:
import dash
from dash import dcc, html, callback, Input, Output
import pandas as pd
import plotly.express as px

In [5]:
# !pip install openpyxl

In [ ]:
# Load sales data
file_path = "./raw/online_retail.xlsx"  # Update this path
df = pd.read_excel(file_path)

___
SALES PER COUNTRY
___

In [ ]:
# Aggregate sales by country
df['Revenue'] = df['Quantity'] * df['UnitPrice']
country_sales = df.groupby('Country', as_index=False).agg({'Revenue': 'sum', 'Quantity': 'sum'})

# Load country coordinates for mapping
geo_data = px.data.gapminder()[['country', 'iso_alpha']].drop_duplicates()
all_countries = pd.DataFrame({'Country': geo_data['country'], 'iso_alpha': geo_data['iso_alpha']})
country_sales = country_sales.merge(all_countries, on='Country', how='left').fillna({'Revenue': 0, 'Quantity': 0})
country_sales = country_sales[country_sales['Country'] != 'Antarctica']

country_sales.head()

,Country,TotalSales,Quantity,iso_alpha
0,Australia,137077.27,83653,AUS
1,Austria,10154.32,4827,AUT
2,Bahrain,548.40,260,BHR
3,Belgium,40910.96,23152,BEL
4,Brazil,1143.60,356,BRA


___
REVENUE OVER TIME
___

In [ ]:
# Aggregate revenue over time
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
time_series = df.resample('M', on='InvoiceDate').agg({'Revenue': 'sum'}).reset_index()
time_series.head()

/var/folders/2j/ryb6v7ns2t9760z6yvvtp83m0000gn/T/ipykernel_40201/3577123508.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  time_series = df.resample('M', on='InvoiceDate').agg({'TotalSales': 'sum'}).reset_index()


,InvoiceDate,TotalSales
0,2010-12-31,748957.020
1,2011-01-31,560000.260
2,2011-02-28,498062.650
3,2011-03-31,683267.080
4,2011-04-30,493207.121


___
MONTHLY CUSTOMER RETENTION
___

In [ ]:


df_last_6_months = df[df['InvoiceDate'] >= six_months_ago].copy()
df_last_6_months['Month'] = df_last_6_months['InvoiceDate'].dt.to_period('M')
df_last_6_months.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSales,Month
220572,556201,23084,RABBIT NIGHT LIGHT,12,2011-06-09 13:01:00,2.08,12347.0,Iceland,24.96,2011-06
220573,556201,23162,REGENCY TEA STRAINER,8,2011-06-09 13:01:00,3.75,12347.0,Iceland,30.00,2011-06
220574,556201,23171,REGENCY TEA PLATE GREEN,12,2011-06-09 13:01:00,1.65,12347.0,Iceland,19.80,2011-06
220575,556201,23172,REGENCY TEA PLATE PINK,12,2011-06-09 13:01:00,1.65,12347.0,Iceland,19.80,2011-06
220576,556201,23170,REGENCY TEA PLATE ROSES,12,2011-06-09 13:01:00,1.65,12347.0,Iceland,19.80,2011-06


In [19]:
customer_months = df_last_6_months.groupby(['CustomerID', 'Month']).size().reset_index(name='Purchases')
customer_months['PreviousMonth'] = customer_months['Month'] - 1

customer_months.head()

,CustomerID,Month,Purchases,PreviousMonth
0,12347.0,2011-06,18,2011-05
1,12347.0,2011-08,22,2011-07
2,12347.0,2011-10,47,2011-09
3,12347.0,2011-12,11,2011-11
4,12348.0,2011-09,3,2011-08


In [20]:
returning_customers = customer_months.merge(
    customer_months, 
    left_on=['CustomerID', 'Month'], 
    right_on=['CustomerID', 'PreviousMonth']
)

returning_customers.head()

,CustomerID,Month_x,Purchases_x,PreviousMonth_x,Month_y,Purchases_y,PreviousMonth_y
0,12362.0,2011-07,19,2011-06,2011-08,33,2011-07
1,12362.0,2011-08,33,2011-07,2011-09,40,2011-08
2,12362.0,2011-09,40,2011-08,2011-10,75,2011-09
3,12362.0,2011-10,75,2011-09,2011-11,19,2011-10
4,12362.0,2011-11,19,2011-10,2011-12,30,2011-11


In [21]:
monthly_retention = returning_customers.groupby('Month_x').agg({'CustomerID': 'nunique'}).reset_index()
monthly_retention.head()

,Month_x,CustomerID
0,2011-06,322
1,2011-07,433
2,2011-08,465
3,2011-09,552
4,2011-10,690
